In [290]:
import logging
import pandas as pd
from datetime import datetime
import country_converter as coco

class Jnry():
    
    def __init__(self, country, path, year=None):
        self.df = pd.read_csv(path)
        self.year = year
       # self.jnry_year = self.get_jnry_year()
        self.country = country
        self.jnry_country = self.get_jnry_country()
        self.party_facts = self.get_party_facts()
        #self.jnry_country = self.get_jnry_country()
        
    def get_jnry_year(self, yr):
        
        if yr is None:
        
            return datetime.now().year
            logging.warning("Input year is missing")
            
        else:
            
            try:
                return parser.parse(str(int(yr))).year
            except:
                return datetime.now().year
                logging.warning("Can't properly parse the year variable")
            
            
    def get_jnry_country(self):
        try:
            return coco.convert(self.country, to='ISO3')
        except:
            return self.country
            logging.warning("Can't properly parse the country variable to ISO3 format")
        
        
    def get_unique_country_year_combinations(self):
        '''returns a dict of the unique year * pol party combinations in the target df, 
        that should be matched with the party facts df'''
        
        return pd.Series(self.df.politicalparty.values,index=self.df.year).to_dict()
        
    def get_party_facts(self):
        PF_URL = "https://partyfacts.herokuapp.com/download/core-parties-csv/"
        pf = pd.read_csv(f"{PF_URL}")
        return pf[pf.loc[:,'country'] == self.jnry_country]
        
    
    def get_party_facts_ids(self):
        
        final_results = []
        for year, polparty in self.get_unique_country_year_combinations().items():
            
            yr = self.get_jnry_year(year)
            
            d = self.party_facts[(self.party_facts.loc[:,'year_last'] >= yr ) | (self.party_facts.loc[:,'year_first'] < yr )] # if the last year that the name was used is AFTER the target date,
            d = d[d.loc[:,'name_short'] == polparty]
    
            results = {'year': year, 
                       'jnry_year' : yr , 
                       'politicalparty': polparty, 
                       'partyfactors_id' : d['partyfacts_id'].values[0],
                       'wikipedia' : d['wikipedia'].values[0]}
    
            final_results.append(results)
        
        return final_results
    
    def merge_party_facts_with_target(self):
        jnry_data = pd.DataFrame(self.get_party_facts_ids())
        jnry = self.df.merge(jnry_data, on=['year', 'politicalparty'])
        return jnry

### First try 

In [292]:
path_to_file = 'https://comparativeagendas.s3.amazonaws.com/datasetfiles/Party_Manifestos_CAP_Web_csv.csv'
a = Jnry(country= 'Spain', path=path_to_file)
df = a.merge_party_facts_with_target()

In [293]:
df.head()

,id,year,politicalparty,majortopic,subtopic,sentence text,jnry_year,partyfactors_id,wikipedia
0,1,1982.0,PSOE,20,2012,Las próximas elecciones generales serán una bu...,1982,1338,https://en.wikipedia.org/wiki/Spanish_Socialis...
1,2,1982.0,PSOE,20,2012,El 28 de octubre al tiempo que se elige a los ...,1982,1338,https://en.wikipedia.org/wiki/Spanish_Socialis...
2,3,1982.0,PSOE,20,2099,La gran mayoría de los españoles es consciente...,1982,1338,https://en.wikipedia.org/wiki/Spanish_Socialis...
3,4,1982.0,PSOE,20,2099,Los ciudadanos han constatado día a día cómo e...,1982,1338,https://en.wikipedia.org/wiki/Spanish_Socialis...
4,5,1982.0,PSOE,20,2011,Por ello cuando las señales de agotamiento se ...,1982,1338,https://en.wikipedia.org/wiki/Spanish_Socialis...
